In [162]:
import json
import numpy as np
import pandas as pd
import geopandas as gp
from plotly import express as px

In [163]:
df = pd.read_json('../data/model_data/data.json')
with open('../map/map.geojson') as file: 
    neighborhoods = json.load(file)

## preliminary cleaning

In [228]:
# drop obviously erroneous data points
df = df[(df['distance_to_CCTC'] < 20) & df['neighborhood'].notnull()]

In [274]:
nbhds = np.append(np.sort(df[df['neighborhood'] != 'Other Surrounding Areas'].neighborhood.unique()), 'Other Surrounding Areas')

mean_price = []
for nbhd in nbhds:
    mean_price.append(round(df[df['neighborhood'] == nbhd].price.mean(), 2))

df_nbhd = pd.concat([pd.DataFrame(nbhds, columns=['neighborhood']), 
                     pd.DataFrame(mean_price, columns=['price'])], axis=1)

In [276]:
# investigate this
df[df['neighborhood'] == 'North Campus'].head()

,address,price,bed,bath,area,company,neighborhood,laundry,pets,parking,...,lon,distance_to_CCTC,distance_to_Pierpont,distance_to_Stadium,distance_to_Union,distance_to_Ross,distance_to_IM,distance_to_NCRB,distance_to_CCRB,distance_to_UgLi
6883,"420 Kellogg Street Ann Arbor, MI 48105",1232.5,1,1.0,650,Friedman Intergrated Real Estate Solutions,North Campus,1.0,1.0,1.0,...,-83.741215,1.755323,1.974067,3.101959,2.005196,1.703930,2.727046,2.197581,1.484511,1.980079
6884,"420 Kellogg Street Ann Arbor, MI 48105",1470.0,2,1.0,650,Friedman Intergrated Real Estate Solutions,North Campus,1.0,1.0,1.0,...,-83.741215,1.755323,1.974067,3.101959,2.005196,1.703930,2.727046,2.197581,1.484511,1.980079
6885,"420 Kellogg Street Ann Arbor, MI 48105",1482.5,2,1.0,650,Friedman Intergrated Real Estate Solutions,North Campus,1.0,1.0,1.0,...,-83.741215,1.755323,1.974067,3.101959,2.005196,1.703930,2.727046,2.197581,1.484511,1.980079
6947,"1780 Broadway Ann Arbor, MI 48105",890.0,4,4.0,None,TDC Management Co.,North Campus,1.0,0.0,1.0,...,-83.721316,2.199767,0.561085,3.969997,2.784566,2.366864,3.674145,0.550299,1.871648,2.541702
7136,"1556a Jones Drive Ann Arbor, MI 48105",1090.0,1,1.0,400,JonesWoods Properties,North Campus,1.0,1.0,1.0,...,-83.731147,1.897875,1.211708,3.548808,2.365993,1.975462,3.209838,1.358837,1.560929,2.210178


##  map of neighborhoods, colored by average price

In [284]:
fig = px.choropleth_mapbox(df_nbhd, geojson=neighborhoods, 
                           color='price', 
                           color_continuous_scale=['green', 'yellow', 'orange', 'red'],
                           mapbox_style="carto-positron",
                           locations='neighborhood',
                           zoom=12.7,
                           opacity=0.7,
                           center={'lat': 42.283497, 'lon': -83.735136},
                           featureidkey='properties.name')

In [285]:
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## map showing location of each "valid" listing

In [288]:
fig = px.scatter_mapbox(df, lat='lat', lon='lon', 
                        hover_name='address', hover_data=['price'],
                        color='neighborhood',
                        color_discrete_sequence=plotly.colors.qualitative.Alphabet,
                        mapbox_style="carto-positron", zoom=12)

In [289]:
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## additional ideas
- neighborhoods by number of listings (density)
- neighborhoods by variability of price
- ???